---
title: "Windpark Kelmarsh (2016) - 12.5 MW"
subtitle: "Reliability Evaluation"
author: "Jorge A. Thomas"
date: April.2025
format:    
    html:
      theme: default
      self-contained: true
      code-fold: true
      df-print: tibble
      code-summary: "Show the code"
      toc: true
      toc-depth: 3
      grid: 
        margin-width: 350px
execute: 
  echo: fenced
reference-location: margin 
citation-location: document
---


## Project Overview

"Kelmarsh Wind Farm is located near Haselbach, Northamptonshire and comprises six 2.05MW Senvion MM92 turbines. The Project was acquired from EON in December 2014 and was constructed in joint venture with Santander using balance sheet finance. Construction was completed in April 2016. It is now owned by Cubico Investments."

-   https://www.blue-energyco.com/our-projects/kelmarsh/

::: {#fig-intro layout-ncol="2"}
![Kelmarsh Wind Farm](../imgs/Kelmarsh_wind_project_blue-energyco.jpg){width="100%"}

![Senvion MM92 turbine](../imgs/turbine-senvion_mm92.jpg){width="100%"}

Near Haselbach, Northamptonshire (UK).
:::


In [ ]:
#| label: Pyhton-setup
#| eval: true
#| echo: false

# https://en.wind-turbine-models.com/turbines/889-senvion-mm92

from pathlib import Path
import duckdb
import polars as pl
import polars.selectors as cs
import plotly.express as px
from great_tables import GT, md

# PROJECT_ROOT = Path(__file__).parent.parent.parent.parent
# folder_path_interim = PROJECT_ROOT / "data" / "interim"
folder_path_interim = "../data/interim/"

Read parquet files


In [ ]:
#| label: Read-Data
tstatus = pl.read_parquet(f"{folder_path_interim}Kelmarsh_Turbines_Status_20160114_20230109_n385133_cols12.parquet")

tstatus

Filter Stop


In [ ]:
#| label: Filter-Stops

stops = pl.sql(    
    """
    SELECT * EXCLUDE ("Custom contract category")    
    FROM tstatus
    WHERE "Status" = 'Stop'
    """).collect()

Filter Forced Outages


In [ ]:
#| label: Filter-Forced-Outages

# Aggregate tables using SQL syntax
foutages = pl.sql(
    """
    SELECT * 
    FROM stops
    WHERE "IEC category" = 'Forced outage' 
    """).collect()

foutages = foutages.rename({"Duration":"TTR"})

# TTF
foutages = foutages.with_columns(
    (pl.col("Timestamp start").shift(-1) - pl.col("Timestamp end")).shift(1).alias("TTF")
    )

foutages = foutages.with_columns(
    pl.when(pl.col("TTF") < 0).then(None).otherwise(pl.col("TTF")).alias("TTF")
)

foutages

## Time Series for Binary Plot


In [ ]:
#| label: Binary-Plot

dtstarts = foutages.select(["Timestamp start", "Status", "Message",  "wt"]).rename({"Timestamp start": "Timestamp"})
dtends = foutages.select(["Timestamp end", "Status", "Message",  "wt"]).with_columns(pl.lit("Start").alias("Status")).rename({"Timestamp end": "Timestamp"})

tstates = pl.concat([dtstarts, dtends]).sort(["wt", 'Timestamp'])

# tstates_ts = dtstarts.join(dtends, on="Timestamp", how="full").sort(["wt", 'Timestamp'])

tstates= tstates.with_columns(
    pl.when(pl.col("Status") == "Stop").then(0).otherwise(1).alias("State"),
    pl.col("Timestamp").dt.year().alias("Year")    
    )

tstates= tstates.with_columns(    
    (pl.col("wt") + "-" + pl.col("Year").cast(str)).alias("wt_Year")
    )

tstates
#TODO: remove missing years!

In [ ]:
#| label: Find-Missing-Values

# Get rows containing any null values
missing = (tstates
          .with_row_count("row_index")
          .filter(pl.any_horizontal(pl.all().is_null()))
          .sort("row_index"))

# Display rows with missing values
print("Rows containing missing values:")
print(missing)

# Get count of missing values per column
print("\nMissing value counts per column:")
print(tstates.null_count())

## Plot Square Signal


In [ ]:
#| label: Square-Signal-Plot

import plotly.express as px

# Other line_shape options, or interpolation methods between given points:

# 'hv' step ends, equivalent to pyplot's post option;
# 'vh' step starts;
# 'hvh' step middles, x axis;
# 'vhv' step middles, y axis;
# 'spline' smooth curve between points;
# 'linear' line segments between points, default value for line_shape.

fig = px.line(tstates, x='Timestamp', y="State", line_shape='hv', facet_row="wt_Year", color="wt",
     title="Time / State Diagram for Kellmarsh Wind Turbines (2016-2022)",
     subtitle= "(0) Down Step=Forced outage (IEC), (1) Up Step=Start",
     labels={"Timestamp": "Time (UTC)", "State": "", "wt": ""},facet_row_spacing=0.01 )

# Update y-axis to show only 0 and 1 as labels
fig.update_yaxes(
    tickvals=[0, 1],  # Set tick values to 0 and 1
    ticktext=["", ""]  # Optional: Explicitly set tick text
)

fig.update_xaxes(matches=None, tickformat="%b"  )


# Update facet labels orientation and size
fig.for_each_annotation(lambda a: a.update(text=a.text.split("-")[-1],textangle=0, xanchor='left', x=-0.04))  # Make labels horizontal
fig.update_annotations(font_size=8)  # Reduce font size

fig.show()